<a href="https://colab.research.google.com/github/juntaozhang/ML/blob/main/helloworld.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# pytorch example

https://github.com/ShusenTang/Dive-into-DL-PyTorch

https://github.com/ShusenTang/Dive-into-DL-PyTorch/blob/master/docs/chapter02_prerequisite/2.2_tensor.md


In [ ]:
# x = torch.empty(5, 3)
# print(x)

# x = torch.rand(5, 3)
# print(x)

# x = torch.zeros(5, 3, dtype=torch.long)
# print(x)

# x = torch.tensor([[5.5, 3],[1.5, 1]])
# print(x)
# print(x.size())
# print(x.shape)

x = torch.ones(3, 3)
y = torch.eye(3)*2
# print(x + y)
# print(torch.add(x, y))
# result = torch.empty(3, 3)
# torch.add(x, y, out=result)
# print(result)
# y.add_(x)
# print(y)
# print(y[:,1])

# x = torch.ones(2, 2, requires_grad=True)
# print(x)
# print(x.grad_fn)

# print(x)
# print(y)
# print(x.mul(y))

data = torch.rand(1, 3, 64, 64)
import torch, torchvision
model = torchvision.models.resnet18(pretrained=True)
labels = torch.rand(1, 1000)
prediction = model(data) # forward pass



In [ ]:
loss = (prediction - labels).sum()
print(loss)
loss.backward() # backward pass

https://pytorch.apachecn.org/docs/1.7/05.html

In [ ]:
import torch

a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)
external_grad = torch.tensor([0.2, 0.8])
Q = 3*a**3 - b**2

print(Q)
print((9*a**2) * external_grad)
print(-2*b * external_grad)

Q.backward(gradient=external_grad)
print(a.grad.data)
print(b.grad)



# OpenCV

In [ ]:
import numpy as np
import cv2 as cv
img = cv.imread('home.jpg')
gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
sift = cv.xfeatures2d.SIFT_create()
kp = sift.detect(gray,None)
img=cv.drawKeypoints(gray,kp,img)
cv.imwrite('sift_keypoints.jpg',img)

# Cifar10

In [ ]:
import torch
from google.colab import drive
drive.mount("/content/drive")
print(torch.cuda.is_available())
%cd /content/drive/MyDrive/ML/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
True
/content/drive/MyDrive/ML


In [ ]:
%mkdir  cifar10/test2_momentum/model

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.cuda.empty_cache()
# import gc
torch.cuda.memory_summary(device=None, abbreviated=False)
# gc.collect()

print(device)
num_epochs = 2
batch_size = 100
learning_rate = 0.001
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

transform = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
            (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)
    )
])
train_dataset = torchvision.datasets.CIFAR10(root='./cifar10', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./cifar10', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
n_total_step = len(train_loader)
print(n_total_step)

model = torchvision.models.vgg16(pretrained=True)
input_lastLayer = model.classifier[6].in_features
model.classifier[6] = nn.Linear(input_lastLayer, 10)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)

for epoch in range(num_epochs):
    for i, (imgs, labels) in enumerate(train_loader):
        imgs = imgs.to(device)
        labels = labels.to(device)

        labels_hat = model(imgs)
        n_corrects = (labels_hat.argmax(axis=1) == labels).sum().item()
        loss_value = criterion(labels_hat, labels)
        loss_value.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(f"epoch {epoch + 1}/{num_epochs}, step: {i + 1}/{n_total_step}: loss = {loss_value:.5f}, acc = {100 * (n_corrects / labels.size(0)):.2f}%")

    torch.save(model, './cifar10/test_vgg16_cifar10_{}.pth'.format(i))

    with torch.no_grad():
        number_corrects = 0
        number_samples = 0
        for i, (test_images_set, test_labels_set) in enumerate(test_loader):
            test_images_set = test_images_set.to(device)
            test_labels_set = test_labels_set.to(device)

            y_predicted = model(test_images_set)
            labels_predicted = y_predicted.argmax(axis=1)
            number_corrects += (labels_predicted == test_labels_set).sum().item()
            number_samples += test_labels_set.size(0)
        print(f"Overall accuracy {(number_corrects / number_samples) * 100}%")

heatmap = pd.DataFrame(data=0, index=classes, columns=classes)
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(batch_size):
            true_label = labels[i].item()
            predicted_label = predicted[i].item()
            heatmap.iloc[true_label, predicted_label] += 1
_, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(heatmap, annot=True, fmt="d", cmap="YlGnBu")
plt.show()


KeyError: ignored

In [ ]:
import time

import torch
import torch.nn as nn
import torchvision
from PIL import Image
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

base_path = "test3"
tensorboard_name = base_path
# 轮次
epoch = 50

labels = {'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}


class MyCifar10(nn.Module):
    def __init__(self):
        super(MyCifar10, self).__init__()
        self.m = nn.Sequential(
                nn.Conv2d(3, 32, (5, 5), padding=2),
                nn.MaxPool2d(2),
                nn.Conv2d(32, 32, (5, 5), padding=2),
                nn.MaxPool2d(2),
                nn.Conv2d(32, 64, (5, 5), padding=2),
                nn.MaxPool2d(2),
                nn.Flatten(),
                nn.Linear(1024, 64),
                nn.Linear(64, 10)
        )

    def forward(self, x):
        x = self.m(x)
        return x


# tensorboard --logdir=logs
writer = SummaryWriter("cifar10/{}/logs".format(base_path))

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("device type: {}".format(device))


def begin_train():
    # 准备 data
    train_data = torchvision.datasets.CIFAR10("./cifar10", train=True, download=True, transform=torchvision.transforms.ToTensor())
    test_data = torchvision.datasets.CIFAR10("./cifar10", train=False, download=True, transform=torchvision.transforms.ToTensor())
    train_data_size = len(train_data)
    test_data_size = len(test_data)
    print("train_data_size:{}".format(train_data_size))
    print("test_data_size:{}".format(test_data_size))
    train_dataloader = DataLoader(dataset=train_data, batch_size=100, drop_last=False)
    test_dataloader = DataLoader(dataset=test_data, batch_size=100, shuffle=True, num_workers=0, drop_last=False)

    # 准备模型
    myModel = MyCifar10()
    myModel.to(device)

    # loss function
    loss_fn = nn.CrossEntropyLoss()
    loss_fn = loss_fn.to(device)

    # 优化器
    learning_rate = 1e-2
    # optimizer = torch.optim.SGD(myModel.parameters(), lr=learning_rate)
    optimizer = torch.optim.SGD(myModel.parameters(), lr=learning_rate, momentum=0.9)

    total_train_step = 0
    train_start_time = time.time()

    for i in range(epoch):
        epoch_start_time = time.time()
        i += 1
        print("-------------Epoch: {} ------------------".format(i))
        myModel.train()
        for data in train_dataloader:
            inputs, target = data
            inputs = inputs.to(device)
            target = target.to(device)
            outputs = myModel(inputs)
            loss = loss_fn(outputs, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_train_step += 1

            if total_train_step % 100 == 0:
                # print("train step: {}, time: {}, loss: {}".format(total_train_step, (time.time() - epoch_start_time), loss.item()))
                writer.add_scalar(tensorboard_name + "_train_loss", loss.item(), total_train_step)
        print("train step: {}".format(total_train_step))

        # 开始测试
        total_loss = 0
        total_accuracy = 0
        myModel.eval()
        with torch.no_grad():
            for data in test_dataloader:
                inputs, target = data
                inputs = inputs.to(device)
                target = target.to(device)
                outputs = myModel(inputs)
                loss = loss_fn(outputs, target)
                total_loss += loss.item()
                accuracy = (outputs.argmax(1) == target).sum()
                total_accuracy += accuracy
        print("test loss: {}".format(total_loss))
        print("test accuracy: {}".format(total_accuracy / test_data_size))
        writer.add_scalar(tensorboard_name + "_test_loss", total_loss, i)
        writer.add_scalar(tensorboard_name + "_test_acc", total_accuracy / test_data_size, i)

        # 保存模型
        torch.save(myModel, "cifar10/{}/model/m_ck_{}".format(base_path, i))
        torch.save(myModel.state_dict(), "cifar10/{}/model/m_state_ck_{}".format(base_path, i))
        print("saved model {}, cost time: {}".format(i, (time.time() - epoch_start_time)))

    print("train cost time: {}".format((time.time() - train_start_time)))


def predict(image_path):
    with torch.no_grad():
        # myModel = torch.load("cifar10/m_ck_3")
        # myModel = torch.load("cifar10/model/m_ck_40", map_location=torch.device('cpu'))

        myModel = MyCifar10()
        # myModel.load_state_dict(torch.load("cifar10/m_state_ck_40"), strict=False)
        myModel.load_state_dict(torch.load("cifar10/{}/model/m_state_ck_5".format(base_path), map_location=torch.device('cpu')), strict=False)

        myModel.to(device)
        myModel.eval()

        image = Image.open(image_path)
        writer.add_images(tensorboard_name + "_old", torchvision.transforms.ToTensor()(image), 1, dataformats="CHW")

        transform = torchvision.transforms.Compose([
            torchvision.transforms.Resize((32, 32)),
            torchvision.transforms.ToTensor()
        ])
        image = transform(image)
        writer.add_images(tensorboard_name + "_new", image, 1, dataformats="CHW")
        # print(image.shape)
        output = myModel(torch.reshape(image, (1, 3, 32, 32)).to(device))
        index = output.argmax(1).item()
        # print(output)
        for k in labels:
            if labels[k] == index:
                print("image: {}, index: {}, label: {}".format(image_path, index, k))
                break


begin_train()
print(labels)
predict(image_path="./cifar10/imgs/dog.png")
predict(image_path="./cifar10/imgs/airplane.png")
predict(image_path="./cifar10/imgs/bird.png")
predict(image_path="./cifar10/imgs/deer.png")
predict(image_path="./cifar10/imgs/horse.png")
predict(image_path="./cifar10/imgs/truck.png")
writer.close()


device type: cuda
Files already downloaded and verified
Files already downloaded and verified
train_data_size:50000
test_data_size:10000
-------------Epoch: 1 ------------------
train step: 500
test loss: 148.6210114955902
test accuracy: 0.4592999815940857
saved model 1, cost time: 13.08716082572937
-------------Epoch: 2 ------------------
train step: 1000
test loss: 127.48356771469116
test accuracy: 0.555899977684021
saved model 2, cost time: 12.790632724761963
-------------Epoch: 3 ------------------


KeyboardInterrupt: ignored